In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName('Writing in Spark') \
.getOrCreate()

25/06/08 03:55:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = [(1,"Alice","Mumbai","2023-01-15",True),
        (1,"Bob","Delhi","2023-04-15",True),
        (1,"Maddu","Channai","2022-06-15",True)]

columns = ["customer_id","name","city","registration_date","is_active"]

In [4]:
df = spark.createDataFrame(data,columns)

In [10]:
df.rdd.getNumPartitions()

2

In [8]:
df.write \
.format('csv').option('header','true')\
.save('/data/write_output_another.csv')

In [9]:
!hadoop fs -head /data/write_output_another.csv

head: `/data/write_output_another.csv': Is a directory


In [11]:
!hadoop fs -ls /data/

Found 3 items
-rw-r--r--   2 root hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 03:55 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 03:58 /data/write_output_another.csv


In [12]:
!hadoop fs -cat /data/write_output_another.csv/*

customer_id,name,city,registration_date,is_active
1,Alice,Mumbai,2023-01-15,true
customer_id,name,city,registration_date,is_active
1,Bob,Delhi,2023-04-15,true
1,Maddu,Channai,2022-06-15,true


Since the RDD has created 2 partitions so there would be two files , so to resolve this we could repartition data.

In [13]:
df.repartition(1).write \
.format('csv').option('header','true')\
.save('/data/write_output_1')

In [16]:
!hadoop fs -ls /data/

Found 4 items
-rw-r--r--   2 root hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 03:55 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-06-08 04:03 /data/write_output_1
drwxr-xr-x   - root hadoop          0 2025-06-08 03:58 /data/write_output_another.csv


In [17]:
!hadoop fs -ls /data/write_output_1

Found 2 items
-rw-r--r--   2 root hadoop          0 2025-06-08 04:03 /data/write_output_1/_SUCCESS
-rw-r--r--   2 root hadoop        141 2025-06-08 04:03 /data/write_output_1/part-00000-92f44d80-1812-48bc-a244-e35752d5052c-c000.csv


In [18]:
# we canb use delimiter as well
df.repartition(1).write \
.format('csv').option('header','true')\
.option('delimiter','|') \
.save('/data/write_output_2')

In [19]:
!hadoop fs -cat /data/write_output_2/*

customer_id|name|city|registration_date|is_active
1|Alice|Mumbai|2023-01-15|true
1|Bob|Delhi|2023-04-15|true
1|Maddu|Channai|2022-06-15|true


In [20]:
spark.stop()